In [25]:
import json
import numpy as np
import csv
from collections import namedtuple

In [26]:
QueryPaperTuple = namedtuple("QueryPaperTuple", ["queryID", "paperID"])

In [27]:
queryList = ['converting text to speech', 'big data', 'efficient estimation of word representations in vector space', 'natural language interface', 'reinforcement learning in video game']
queryToIdx = {queryList[i]:i for i in range(len(queryList))}
queryToIdx['converting word to speech'] = 0
queryToIdx['Big data'] = 1

In [28]:
fileName = 'annotated2.csv'     ## annotated.csv  entityAnnotations.csv entityAnnotationsSiamese.csv
filePath = './data/'
annotationFile = filePath + fileName
dupDetectionDict = dict()
with open (annotationFile,'r') as csv_file:
    reader =csv.reader(csv_file)
    next(reader) # skip first row
    for row in reader:
        queryID = queryToIdx[row[0]]
        paperID = row[1]
        score = row[-1]
        dupDetectionDict[QueryPaperTuple(queryID, paperID)] = score

In [29]:
fileName = 'entityAnnotations2.csv'     ## annotated.csv  entityAnnotations.csv entityAnnotationsSiamese.csv
filePath = './data/'
#dupDetectionDict = dict()
annotationFile = filePath + fileName
with open (annotationFile,'r') as csv_file:
    reader =csv.reader(csv_file)
    next(reader) # skip first row
    for row in reader:
        queryID = queryToIdx[row[0]]
        paperID = row[1]
        score = row[-1]
        if QueryPaperTuple(queryID, paperID) in dupDetectionDict and score != dupDetectionDict[QueryPaperTuple(queryID, paperID)]:
            print('discrepancy', queryID, paperID, score, dupDetectionDict[QueryPaperTuple(queryID, paperID)])
        else:
            dupDetectionDict[QueryPaperTuple(queryID, paperID)] = score

In [30]:
fileName = 'entityAnnotationsSiamese.csv'     ## annotated.csv  entityAnnotations.csv entityAnnotationsSiamese.csv
filePath = './data/'
annotationFile = filePath + fileName
with open (annotationFile,'r') as csv_file:
    reader =csv.reader(csv_file)
    next(reader) # skip first row
    for row in reader:
        queryID = queryToIdx[row[0]]
        paperID = row[1]
        score = row[-1]
        if QueryPaperTuple(queryID, paperID) in dupDetectionDict and score != dupDetectionDict[QueryPaperTuple(queryID, paperID)]:
            print('discrepancy', queryID, paperID, score, dupDetectionDict[QueryPaperTuple(queryID, paperID)])
        else:
            dupDetectionDict[QueryPaperTuple(queryID, paperID)] = score

In [41]:
with open('./data/annotations.qrel', 'w') as outfile:
    for queryPaperTuple in dupDetectionDict.keys():
        writeString = str(queryPaperTuple.queryID) + ' '+ '0' + ' ' + queryPaperTuple.paperID + ' '+ str(dupDetectionDict[queryPaperTuple])
        outfile.write(writeString)
        outfile.write('\n')

In [43]:
queryList = ['converting text to speech', 'Big data', 'efficient estimation of word representations in vector space', 'natural language interface', 'reinforcement learning in video game']
queryToIdx = {queryList[i]:i for i in range(len(queryList))}
annotationDict = [{} for i in range(len(queryList) + 1)]    # + 1 for 1 based indexing in
with open('./data/annotations.qrel') as file:
    for line in file:
        lineString = line.split()
        qid = int(lineString[0])
        docno = lineString[2]
        relScore = int(lineString[-1])
        annotationDict[qid][docno] = relScore

In [45]:
def dcg(relevanceScores, k = 10, method=0):
    """
    Returns discounted cumulative gain (dcg)
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    relevanceScores = np.asfarray(relevanceScores)[:k]
    if relevanceScores.size:
        if method == 0:
            return relevanceScores[0] + np.sum(relevanceScores[1:] / np.log2(np.arange(2, relevanceScores.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0

def ndcgMax(relevanceScores, k=10, method=0):
    return dcg(sorted(relevanceScores, reverse=True), k, method)

def ndcg(relevanceScores, ndcgMax, k = 10, method=0):
    return dcg(relevanceScores, k, method) / ndcgMax
    

In [50]:
ndcgMaxPerQuery = []
for i in range(len(queryList)):
    ndcgMaxPerQuery.append(ndcgMax(list(annotationDict[i].values())))

In [52]:
relevanceScores= []
### search_results.json 5, entity_search_results.json 6, entity_search_resultsSiamese.json 1
filePath = './data/'

fileName = 'entity_search_resultsSiamese.json'
annotationFile = filePath + fileName

with open(annotationFile, 'r') as file:
    for line in file:
        data = json.loads(line)
        result = data['result']
        for i in range(len(result)):
            result[i] = [annotationDict[i].get(ID, 0) for ID in result[i]]
        relevanceScores.append(result)
results = []
meanScores = []

for i in range(len(relevanceScores)):
    relScoresForThisMethod = relevanceScores[i]
    ndcgScoresMethod = []
    for q in range(len(queryList)):
        ndcgScoresMethod.append(ndcg(relScoresForThisMethod[q], ndcgMaxPerQuery[q]))
    results.append(ndcgScoresMethod)
    meanScores.append(np.mean(ndcgScoresMethod))

In [53]:
print(meanScores)

[0.9879925699387805]


In [54]:
for result in results:
    result = [round(i, 2) for i in result]
    print(result)

[1.0, 1.0, 1.0, 1.0, 0.94]
